In [65]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime
import torch
import random


In [66]:


purchase_df = pd.read_csv('robi-datathon-30-qualifier-problem-3/purchase.csv')
boxes_df = pd.read_csv('robi-datathon-30-qualifier-problem-3/boxes.csv')

In [67]:
purchase_df.head()

,PURCHASE_DATE,MAGIC_KEY,BOX_ID,BOX_COUNT
0,1/2/2019,2CED678A247,12.0,1.0
1,1/2/2019,2BF58D91BA1,12.0,1.0
2,1/2/2019,2C15B86534E,99.0,1.0
3,1/2/2019,2C32D9A859A,6.0,1.0
4,1/2/2019,2C7A55404D1,4.0,1.0


In [68]:
base_date = datetime.date(2018, 10, 1)
purchase_df['day'] = purchase_df['PURCHASE_DATE'].apply(lambda x: (datetime.datetime.strptime(x, '%d/%m/%Y').date() - base_date).days)



In [69]:
purchase_df.sample(10)

,PURCHASE_DATE,MAGIC_KEY,BOX_ID,BOX_COUNT,day
2186910,8/10/2018,292A1818549,247.0,1.0,7
314065,16/2/2019,2C16D8245E1,130.0,1.0,138
1044656,28/1/2019,2CAD8A40A0E,106.0,1.0,119
1906929,22/11/2018,2CCABEFBFA2,89.0,1.0,52
1147001,2/12/2018,2BCA84A1B8E,143.0,1.0,62
558019,28/2/2019,2CD574E8AB9,119.0,1.0,150
87741,5/2/2019,2C2B08A4208,94.0,1.0,127
1763381,8/11/2018,28DAF80ECDB,255.0,1.0,38
676709,5/1/2019,2BDFDE3FBA6,106.0,1.0,96
58683,3/2/2019,2BF4F656140,119.0,1.0,125


In [70]:
purchase_df.drop(['PURCHASE_DATE'], axis=1, inplace=True)

In [71]:
customer_keys = purchase_df['MAGIC_KEY'].unique()
customer_ids = {customer_keys[i]: i for i in range(len(customer_keys))}

In [72]:
purchase_df.drop(purchase_df[purchase_df['BOX_ID'].isnull()].index, inplace=True)
purchase_df.drop(purchase_df[purchase_df['BOX_COUNT'] < 0].index, inplace=True)

In [73]:
purchase_df['BOX_COUNT'] = purchase_df['BOX_COUNT'].astype(int)
purchase_df['BOX_ID'] = purchase_df['BOX_ID'].astype(int)

In [74]:
purchase_df.head(10)

,MAGIC_KEY,BOX_ID,BOX_COUNT,day
0,2CED678A247,12,1,123
1,2BF58D91BA1,12,1,123
2,2C15B86534E,99,1,123
3,2C32D9A859A,6,1,123
4,2C7A55404D1,4,1,123
5,29D969045C2,238,1,123
6,28E5EA49074,227,1,123
7,2CEFA3A8659,6,1,123
8,2A00DE30F46,204,1,123
9,291C04B5CBF,231,1,123


In [75]:
len(purchase_df['MAGIC_KEY'].unique())

1274087

In [76]:
len(purchase_df)

2455800

In [77]:
purchases = [[] for _ in range(len(customer_keys))]
purchase_df.apply(lambda x: purchases[customer_ids[x['MAGIC_KEY']]].append((x['BOX_ID'], x['BOX_COUNT'], x['day'])), axis=1)

0          None
1          None
2          None
3          None
4          None
           ... 
2455859    None
2455860    None
2455861    None
2455862    None
2455863    None
Length: 2455800, dtype: object

In [78]:
prob3_df =pd.read_csv('robi-datathon-30-qualifier-problem-3/problem 3.csv')
prob3_df.head()

,MAGIC_KEY
0,2BCFE9C06A7
1,2C2A872B5A2
2,2C6A897671B
3,2C6F1287F53
4,2C658198CC9


In [79]:
len(prob3_df)

5379

In [80]:
prob3_customers = [customer_ids[x] for x in prob3_df['MAGIC_KEY'].unique()]
len(prob3_customers)

5379

In [81]:
for i, _ in enumerate(purchases):
    purchases[i].sort(key=lambda x: x[2])

In [82]:
a = 10
b = 10
print("single purchase: ")
for i, p in enumerate(purchases):
    if len(p) == 1:
        if a > 0:
            print(i, p)
            a -= 1

print("more than 5 purchase: ")
for i, p in enumerate(purchases):
    if len(p) > 5:
        if b > 0:
            print(i, p)
            b -= 1


single purchase: 
0 [(12, 1, 123)]
1 [(12, 1, 123)]
3 [(6, 1, 123)]
4 [(4, 1, 123)]
6 [(227, 1, 123)]
7 [(6, 1, 123)]
10 [(12, 1, 123)]
11 [(231, 1, 123)]
13 [(278, 1, 123)]
16 [(231, 1, 123)]
more than 5 purchase: 
5 [(238, 1, 14), (238, 1, 36), (238, 1, 56), (238, 1, 80), (238, 1, 100), (238, 1, 123)]
14 [(203, 1, 35), (203, 1, 53), (203, 1, 63), (221, 1, 72), (203, 1, 81), (203, 1, 87), (238, 1, 97), (204, 1, 123), (204, 1, 135), (272, 1, 150)]
38 [(8, 1, 15), (8, 1, 30), (8, 1, 60), (8, 1, 82), (8, 1, 104), (8, 1, 123)]
39 [(238, 1, 20), (238, 1, 44), (268, 1, 54), (238, 1, 67), (238, 1, 76), (238, 1, 89), (238, 1, 98), (238, 1, 105), (228, 1, 123), (238, 1, 129), (238, 1, 133), (238, 1, 138), (238, 1, 142), (230, 1, 147), (238, 1, 150)]
42 [(260, 1, 8), (203, 1, 23), (221, 1, 38), (221, 1, 65), (271, 1, 84), (272, 1, 104), (221, 1, 123), (276, 1, 145)]
43 [(137, 1, 6), (137, 1, 23), (137, 1, 42), (137, 1, 62), (137, 1, 79), (174, 1, 97), (174, 1, 123)]
44 [(238, 1, 18), (238, 1, 1

In [83]:
boxes_df.head(10)

,BOX_ID,QUALITY,DELIVERY_OPTION,MILK,MEAT,UNIT_PRICE
0,1,Premium,Home Delivery - CoD,0.0,2.7,9.96
1,2,Premium,Home Delivery - CoD,0.0,2.3,11.96
2,3,Premium,Home Delivery - CoD,0.0,2.4,11.96
3,4,Premium,Home Delivery - CoD,0.0,2.5,11.96
4,5,Premium,Home Delivery - CoD,0.0,2.6,11.96
5,6,Premium,Home Delivery - CoD,0.0,2.7,11.96
6,7,Premium,Home Delivery - CoD,0.0,2.8,11.96
7,8,Premium,Home Delivery - CoD,10.0,0.0,12.18
8,9,Premium,Home Delivery - CoD,10.5,0.0,12.78
9,10,Premium,Home Delivery - CoD,8.0,1.3,12.98


In [84]:
len(boxes_df)

290

In [85]:
def meat_consumed(cid, start_day, end_day):
    boxes = filter(lambda x: x[2] >= start_day and x[2] < end_day, purchases[cid])
    meat_consumed = 0
    for id, cnt, _ in boxes:
        meat_consumed += cnt * boxes_df.at[id - 1, 'MEAT']
    return meat_consumed

def milk_consumed(cid, start_day, end_day):
    boxes = filter(lambda x: x[2] >= start_day and x[2] < end_day, purchases[cid])
    milk_consumed = 0
    for id, cnt, _ in boxes:
        milk_consumed += cnt * boxes_df.at[id - 1, 'MILK']
    return milk_consumed


def meat_consumption_rate(cid):
    end_day = 151
    consumptions = list(filter(lambda x: x[2] < end_day and boxes_df.at[x[0] - 1, 'MEAT'] > 0.0, purchases[cid]))
    if len(consumptions) == 0:
        return 0.0
    start = consumptions[0][2]
    return meat_consumed(cid, start, end_day) / (end_day - start)

def milk_consumption_rate(cid):
    end_day = 151
    consumptions = list(filter(lambda x: x[2] < end_day and boxes_df.at[x[0] - 1, 'MILK'] > 0.0, purchases[cid]))
    if len(consumptions) == 0:
        return 0.0
    start = consumptions[0][2]
    return milk_consumed(cid, start, end_day) / (end_day - start)

In [86]:
last_day = purchase_df['day'].max()
last_day

150

In [87]:
last_date = datetime.date(2019, 2, 28)
(last_date - base_date).days

150

In [88]:
first_day = purchase_df['day'].min()
first_day

0

In [89]:
meat_consumption_rate(0)

0.05357142857142857

In [90]:
def last_meat_consumption(cid, end_day=151):
    consumptions = list(filter(lambda x: x[2] < end_day and boxes_df.at[x[0] - 1, 'MEAT'] > 0.0, purchases[cid]))
    try:
        bid, cnt, day = consumptions[-1]
    except:
        return None
    return [cnt * boxes_df.at[bid - 1, 'MEAT'], 
            end_day - day]
last_meat_consumption(3588, 150)

[4.4, 27]

In [91]:
split = int(len(purchases) * 0.1)
idx = [i for i in range(len(purchases))]
random.shuffle(idx)
val_idx = idx[:split]
train_idx = idx[split:]

def generate_batch(day, batch_size, split='train'):
    batch = []
    labels = []
    n = 0
    if split == 'train':
        random.shuffle(train_idx)
        current_idx = train_idx
    else:
        random.shuffle(val_idx)
        current_idx = val_idx
    for cid in current_idx:
        if len(purchases[cid]) < 2:
            continue
        last_meat = last_meat_consumption(cid, day)
        if last_meat is not None:
            rate1 = meat_consumption_rate(cid)
            rate2 = milk_consumption_rate(cid)
            next_meat = meat_consumed(cid, day, day + 15)
            if next_meat > 0.0:
                features = [rate1, rate2]
                features.extend(last_meat)
                batch.append(features)
                labels.append(next_meat)
                n += 1
                if split=='train' and n == batch_size:
                    return batch, labels
    if split == 'train':
        return None, None
    else:
        return batch, labels
batch, labels = generate_batch(15, 10)
print(batch)
print(labels)

[[0.024657534246575342, 0.2191780821917808, 1.8, 10], [0.09337748344370862, 0.6754966887417219, 1.8, 15], [0.024657534246575342, 0.2191780821917808, 1.8, 10], [0.07655172413793103, 0.0, 3.6, 9], [0.03377483443708609, 0.10596026490066225, 1.8, 15], [0.07567567567567568, 0.0, 2.7, 12], [0.09375, 0.625, 1.3, 8], [0.1221476510067114, 0.3221476510067114, 1.8, 13], [0.03768115942028986, 0.2971014492753623, 1.6, 2], [0.08169014084507044, 0.676056338028169, 1.8, 6]]
[1.8, 1.8, 1.8, 3.5, 3.3, 3.6, 1.8, 4.0, 1.8, 1.8]


In [92]:
import gc

class MeatPredictor(torch.nn.Module):
    def __init__(self):
        super(MeatPredictor, self).__init__()
        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(4, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 1), 
            torch.nn.ReLU()
        )
    def forward(self, x):
        return self.mlp(x).reshape(-1)

model = MeatPredictor().cuda()
model(torch.tensor(batch, dtype=torch.float32).cuda())

tensor([1.5466, 2.4872, 1.5466, 1.1121, 2.4628, 1.7867, 1.2673, 2.1026, 0.0975,
        0.8413], device='cuda:0', grad_fn=<ViewBackward0>)

In [93]:


torch.cuda.empty_cache()
gc.collect()

loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
best_loss = 1e9
batch_size = 64
epochs = 2000

val, val_labels = generate_batch(136, split, 'val')
val = torch.tensor(val, dtype=torch.float32).cuda()
val_labels = torch.tensor(val_labels, dtype=torch.float32).cuda()

for epoch in range(epochs):
    batch, labels = generate_batch(random.randint(70, 136), batch_size)
    if batch is None:
        continue
    batch = torch.tensor(batch, dtype=torch.float32).cuda()
    labels = torch.tensor(labels, dtype=torch.float32).cuda()
    optimizer.zero_grad()
    outputs = model(batch)
    loss = loss_fn(outputs, labels)
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(epoch, loss.item())
    if epoch % 20 == 0:
        val_outputs = model(val)
        val_loss = loss_fn(val_outputs, val_labels)
        print("Validation loss: ", val_loss.item())
        if val_loss.item() < best_loss:
            torch.save(model.state_dict(), 'meat_predictor.pth')
            best_loss = val_loss.item()

model.load_state_dict(torch.load('meat_predictor.pth'))

0 9.38676929473877
Validation loss:  35.77566146850586
10 12.606657981872559
20 2.4572548866271973
Validation loss:  3.8805131912231445
30 3.5723400115966797
40 3.325395345687866
Validation loss:  3.659686803817749
50 3.0149009227752686
60 1.9966638088226318
Validation loss:  3.783442974090576
70 1.3174002170562744
80 2.331120491027832
Validation loss:  2.9084737300872803
90 1.8997175693511963
100 3.1019058227539062
Validation loss:  2.7042176723480225
110 1.5167392492294312
120 1.3443533182144165
Validation loss:  2.0388729572296143
130 1.6298325061798096
140 0.925134003162384
Validation loss:  1.6237194538116455
150 0.8593209981918335
160 1.2267444133758545
Validation loss:  1.5867784023284912
170 0.5649392604827881
180 0.4686717987060547
Validation loss:  0.9630466103553772
190 0.22463130950927734
200 0.5144667029380798
Validation loss:  0.733742892742157
210 0.22954094409942627
220 0.4511116147041321
Validation loss:  0.6748250722885132
230 0.24186798930168152
240 0.624718189239502

<All keys matched successfully>

In [94]:
print("best loss: ", best_loss)

best loss:  0.4750223755836487


In [95]:
def predict(cid, day):
    rate1 = meat_consumption_rate(cid)
    rate2 = milk_consumption_rate(cid)
    last_meat = last_meat_consumption(cid, day)
    if last_meat is not None:
        features = [rate1, rate2]
        features.extend(last_meat)
        return model(torch.tensor(features, dtype=torch.float32).cuda()).item()
    return 0.0

In [96]:

data = []
for cid in prob3_customers:
    data.append({
        'MAGIC_KEY': customer_keys[cid],
        'MEAT': predict(cid, 151)
    })
sub_df = pd.DataFrame(data, columns=['MAGIC_KEY', 'MEAT'])
sub_df.to_csv('submission.csv', index=False)